# polars学习簿

polars是一个高性能的 DataFrame 库，专为处理大规模数据而设计。它提供了类似于 pandas的接口，但在性能和内存效率方面有显著优势。

polars的核心结构是 Series 和 DataFrame 数据结构，支持惰性执行和并行处理，能够高效处理海量数据。其底层使用 Rust 编写，并通过 PyO3 提供 Python 绑定，结合了 Rust 的性能优势和 Python 的易用性。

polars采用 Apache 2.0 开源许可证，由 Ritchie Vink 创建并维护。它的设计目标是为数据科学和数据分析提供快速、可靠的工具，尤其适合处理超出内存限制的大型数据集。

下面是 polars的主要功能及特点：

- ​高性能​​：底层由 Rust 实现，支持多线程并行处理，性能远超纯 Python 实现的数据处理库。
- ​惰性执行​​：支持惰性计算模式，可以优化整个查询计划，减少不必要的计算和内存使用。
- ​内存效率​​：使用 Apache Arrow 作为内存格式，支持零拷贝操作和高效的内存管理。
- ​丰富的 API​​：提供类似 pandas的接口，同时支持表达式 API，能够进行复杂的数据变换和聚合操作。
- ​类型安全​​：支持严格的数据类型系统，避免常见的数据类型错误。
- ​集成能力​​：可以轻松与其他数据科学库（如 numpy、pandas）集成，支持多种数据格式的读写。

这份工作簿将带你快速入门 polars，并介绍其基本功能和用法。在学习和使用的时候可以对比pandas一起学习。更高级的特性可能需要你在实际应用中进一步探索。



## 准备
你可以通过 pip来安装 polars：

In [2]:
pip install polars

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


然后就可以在你的程序中引入 polars库：

In [3]:
import polars as pl

下面所有的操作都假定你已经完成了上面两步。
接下来，让我们从基础着手认识polars。

## 核心数据类型与结构
### Series
Series是 polars中的一维数据结构，类似于 pandas的 Series或 numpy的 ndarray。  
在一个 Series 中，所有元素都具有相同的数据类型。

In [4]:
# 创建 Series
s = pl.Series("ages", [25, 30, 35])
print(s)

shape: (3,)
Series: 'ages' [i64]
[
	25
	30
	35
]


在创建序列时，Polars 会根据提供的值来推断数据类型。你也可以指定具体的数据类型来覆盖这种推断机制。

In [5]:
s1 = pl.Series("ints", [1, 2, 3, 4, 5])
s2 = pl.Series("uints", [1, 2, 3, 4, 5], dtype=pl.UInt64)
print(s1.dtype, s2.dtype)

Int64 UInt64


`name`获取数组名,`dtype` 则表示数组的数据类型.

In [6]:
# 基本属性
print(f"Name: {s.name}")
print(f"Length: {len(s)}")
print(f"Data type: {s.dtype}")
print(f"Values: {s.to_list()}")

Name: ages
Length: 3
Data type: Int64
Values: [25, 30, 35]


`series`可直接求最大值，最小值，平均值和总和。

In [7]:
# 基本操作
print(f"Max: {s.max()}")
print(f"Min: {s.min()}")
print(f"Mean: {s.mean()}") 
print(f"Sum: {s.sum()}")

Max: 35
Min: 25
Mean: 30.0
Sum: 90


`Series`支持向量间的基本运算，比如加减乘除等。

In [8]:
x = pl.Series("temp", [2, 3, 4])

In [9]:
# 向量化操作
print(s + x)
print(s - x)
print(s * x)
print(s / x)

shape: (3,)
Series: 'ages' [i64]
[
	27
	33
	39
]
shape: (3,)
Series: 'ages' [i64]
[
	23
	27
	31
]
shape: (3,)
Series: 'ages' [i64]
[
	50
	90
	140
]
shape: (3,)
Series: 'ages' [f64]
[
	12.5
	10.0
	8.75
]


## DataFrame 
DataFrame是 polars的核心数据结构，它是一个二维表格，类似于 pandas的 DataFrame或关系数据库中的表。

### 创建 DataFrame

支持从多种数据源创建 DataFrame，比如列表、字典或直接读取文件。

In [10]:
# 从字典创建 DataFrame
df = pl.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "age": [25, 30, 35, 40, 45],
    "city": ["New York", "London", "Paris", "Tokyo", "Berlin"],
    "salary": [5000, 6000, 7000, 8000, 9000]
})
print(df)
df.write_csv("./output.csv")

shape: (5, 4)
┌─────────┬─────┬──────────┬────────┐
│ name    ┆ age ┆ city     ┆ salary │
│ ---     ┆ --- ┆ ---      ┆ ---    │
│ str     ┆ i64 ┆ str      ┆ i64    │
╞═════════╪═════╪══════════╪════════╡
│ Alice   ┆ 25  ┆ New York ┆ 5000   │
│ Bob     ┆ 30  ┆ London   ┆ 6000   │
│ Charlie ┆ 35  ┆ Paris    ┆ 7000   │
│ David   ┆ 40  ┆ Tokyo    ┆ 8000   │
│ Eve     ┆ 45  ┆ Berlin   ┆ 9000   │
└─────────┴─────┴──────────┴────────┘


### 查看DataFrame  

使用`describe`函数来计算数据框中所有列的汇总统计信息:

In [11]:
print(df.describe())

shape: (9, 5)
┌────────────┬───────┬──────────┬────────┬────────────┐
│ statistic  ┆ name  ┆ age      ┆ city   ┆ salary     │
│ ---        ┆ ---   ┆ ---      ┆ ---    ┆ ---        │
│ str        ┆ str   ┆ f64      ┆ str    ┆ f64        │
╞════════════╪═══════╪══════════╪════════╪════════════╡
│ count      ┆ 5     ┆ 5.0      ┆ 5      ┆ 5.0        │
│ null_count ┆ 0     ┆ 0.0      ┆ 0      ┆ 0.0        │
│ mean       ┆ null  ┆ 35.0     ┆ null   ┆ 7000.0     │
│ std        ┆ null  ┆ 7.905694 ┆ null   ┆ 1581.13883 │
│ min        ┆ Alice ┆ 25.0     ┆ Berlin ┆ 5000.0     │
│ 25%        ┆ null  ┆ 30.0     ┆ null   ┆ 6000.0     │
│ 50%        ┆ null  ┆ 35.0     ┆ null   ┆ 7000.0     │
│ 75%        ┆ null  ┆ 40.0     ┆ null   ┆ 8000.0     │
│ max        ┆ Eve   ┆ 45.0     ┆ Tokyo  ┆ 9000.0     │
└────────────┴───────┴──────────┴────────┴────────────┘


使用`df.shape`和`df.columns`属性查看DataFrame的形状和列名。

In [12]:
# 基本属性
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns}")

Shape: (5, 4)
Columns: ['name', 'age', 'city', 'salary']


`df.schema`返回一个包含所有列名及其对应数据类型的字典，可以使用`schema`
来查看数据框的结构。

In [13]:
print(f"Schema: {df.schema}")

Schema: Schema([('name', String), ('age', Int64), ('city', String), ('salary', Int64)])


而 `df.dtypes`返回一个仅包含所有列数据类型的列表。

In [14]:
print(f"Data types: {df.dtypes}")

Data types: [String, Int64, String, Int64]


与`Series`类似，创建`DataFrame`时，Polars会自动推断其结构可以使用`schema`和`schema_overrides`覆盖某些列的推理结果。  

使用`schema`时，对于不指定覆盖名称的列，需要使用None表示。

In [15]:
df0 = pl.DataFrame(
    {
        "name": ["Alice", "Ben", "Chloe", "Daniel"],
        "age": [27, 39, 41, 43],
    },
    schema={"name": None, "age": pl.UInt8},
)

print(df0)

shape: (4, 2)
┌────────┬─────┐
│ name   ┆ age │
│ ---    ┆ --- │
│ str    ┆ u8  │
╞════════╪═════╡
│ Alice  ┆ 27  │
│ Ben    ┆ 39  │
│ Chloe  ┆ 41  │
│ Daniel ┆ 43  │
└────────┴─────┘


而`schema overrides`会更加方便，因为它允许直接忽略不想覆盖的列。

In [16]:
df0 = pl.DataFrame(
    {
        "name": ["Alice", "Ben", "Chloe", "Daniel"],
        "age": [27, 39, 41, 43],
    },
    schema_overrides={"age": pl.UInt8},
)

print(df0)

shape: (4, 2)
┌────────┬─────┐
│ name   ┆ age │
│ ---    ┆ --- │
│ str    ┆ u8  │
╞════════╪═════╡
│ Alice  ┆ 27  │
│ Ben    ┆ 39  │
│ Chloe  ┆ 41  │
│ Daniel ┆ 43  │
└────────┴─────┘


`head`函数头部显示`DataFrame`的前几行。
默认情况下，会显示前 5 行，但也可以指定想要显示的行数。  
`tail`函数用于返回`DataFrame`的最后n行，类似于pandas中的tail。  
`sample`函数用于从`DataFrame`中随机抽取n行，不重复（默认情况下，除非指定允许重复）。

In [17]:
# 查看数据
print("Head:")
print(df.head(2))
print("Tail:")
print(df.tail(2))
print("Sample:")
print(df.sample(2))

Head:
shape: (2, 4)
┌───────┬─────┬──────────┬────────┐
│ name  ┆ age ┆ city     ┆ salary │
│ ---   ┆ --- ┆ ---      ┆ ---    │
│ str   ┆ i64 ┆ str      ┆ i64    │
╞═══════╪═════╪══════════╪════════╡
│ Alice ┆ 25  ┆ New York ┆ 5000   │
│ Bob   ┆ 30  ┆ London   ┆ 6000   │
└───────┴─────┴──────────┴────────┘
Tail:
shape: (2, 4)
┌───────┬─────┬────────┬────────┐
│ name  ┆ age ┆ city   ┆ salary │
│ ---   ┆ --- ┆ ---    ┆ ---    │
│ str   ┆ i64 ┆ str    ┆ i64    │
╞═══════╪═════╪════════╪════════╡
│ David ┆ 40  ┆ Tokyo  ┆ 8000   │
│ Eve   ┆ 45  ┆ Berlin ┆ 9000   │
└───────┴─────┴────────┴────────┘
Sample:
shape: (2, 4)
┌───────┬─────┬────────┬────────┐
│ name  ┆ age ┆ city   ┆ salary │
│ ---   ┆ --- ┆ ---    ┆ ---    │
│ str   ┆ i64 ┆ str    ┆ i64    │
╞═══════╪═════╪════════╪════════╡
│ Eve   ┆ 45  ┆ Berlin ┆ 9000   │
│ David ┆ 40  ┆ Tokyo  ┆ 8000   │
└───────┴─────┴────────┴────────┘


`glimpse`是另一个用于展示数据框前几行值的函数，但其输出格式与`head`不同。在这里，输出中的每一行对应一个单独的列，这使得对更宽的数据框进行检查变得更加容易:

In [18]:
print(df.glimpse(return_as_string=True))

Rows: 5
Columns: 4
$ name   <str> 'Alice', 'Bob', 'Charlie', 'David', 'Eve'
$ age    <i64> 25, 30, 35, 40, 45
$ city   <str> 'New York', 'London', 'Paris', 'Tokyo', 'Berlin'
$ salary <i64> 5000, 6000, 7000, 8000, 9000



## 表达式(Expression)
`Polars` 已经开发出了自己的特定领域语言(DSL)来处理数据转换。这种语言非常易于使用，并且能够处理复杂的查询，同时这些查询仍能保持较高的可读性。  

这里将要介绍的表达式(Expression)和后续介绍的执行环境(Context)对于实现这种可读性非常重要，同时还能让`Polars` 查询引擎优化查询，使其运行速度尽可能快。

表达式是一种对数据转换的延迟表示形式。其具有模块化和灵活性的特点，这意味着可以将它们用作构建更复杂表达式的构建块。一个计算人体BMI的`Polars` 示例表达式如下:

In [19]:
pl.col("weight") / (pl.col("height") ** 2)

<Expr ['[(col("weight")) / (col("heigh…'] at 0x1E8AC29AF10>

进一步的，为了方便，可以用变量存储表达式。

In [20]:
bmi_expr = pl.col("weight") / (pl.col("height") ** 2)
print(bmi_expr)

[(col("weight")) / (col("height").pow([dyn int: 2]))]


## 执行环境(Context)
因为表达式是延迟执行的，所以目前尚未进行任何计算。这就是我们需要使用执行环境的原因所在。

Polars 表达式需要一个执行环境才能产生结果。根据其使用的语境不同，同一个“Polars”表达式可能会产生不同的结果。结果。在本节中，我们将了解 Polars所提供的四种最常见的应用场景，包括：`select, with_column, filter, group_by`。

我们将使用下列`Datafreme`作为示例。

In [21]:
from datetime import date

df = pl.DataFrame(
    {
        "name": ["Alice Archer", "Ben Brown", "Chloe Cooper", "Daniel Donovan"],
        "birthdate": [
            date(1997, 1, 10),
            date(1985, 2, 15),
            date(1983, 3, 22),
            date(1981, 4, 30),
        ],
        "weight": [57.9, 72.5, 53.6, 83.1],  # (kg)
        "height": [1.56, 1.77, 1.65, 1.75],  # (m)
    }
)

print(df)

shape: (4, 4)
┌────────────────┬────────────┬────────┬────────┐
│ name           ┆ birthdate  ┆ weight ┆ height │
│ ---            ┆ ---        ┆ ---    ┆ ---    │
│ str            ┆ date       ┆ f64    ┆ f64    │
╞════════════════╪════════════╪════════╪════════╡
│ Alice Archer   ┆ 1997-01-10 ┆ 57.9   ┆ 1.56   │
│ Ben Brown      ┆ 1985-02-15 ┆ 72.5   ┆ 1.77   │
│ Chloe Cooper   ┆ 1983-03-22 ┆ 53.6   ┆ 1.65   │
│ Daniel Donovan ┆ 1981-04-30 ┆ 83.1   ┆ 1.75   │
└────────────────┴────────────┴────────┴────────┘


### select
`select`基于表达式的列进行操作。这种选择操作可能生成新的列，这些列可以是聚合结果、其他列的组合或者是常量值。

In [22]:
result = df.select(
    bmi=bmi_expr,
    avg_bmi=bmi_expr.mean(),
    ideal_max_bmi=25,
)
print(result)

shape: (4, 3)
┌───────────┬───────────┬───────────────┐
│ bmi       ┆ avg_bmi   ┆ ideal_max_bmi │
│ ---       ┆ ---       ┆ ---           │
│ f64       ┆ f64       ┆ i32           │
╞═══════════╪═══════════╪═══════════════╡
│ 23.791913 ┆ 23.438973 ┆ 25            │
│ 23.141498 ┆ 23.438973 ┆ 25            │
│ 19.687787 ┆ 23.438973 ┆ 25            │
│ 27.134694 ┆ 23.438973 ┆ 25            │
└───────────┴───────────┴───────────────┘


在`select`中的表达式必须生成长度完全相同的序列，否则必须生成一个标量值。

标量值会进行扩展处理，以与剩余序列的长度相匹配。像上述所使用的数字这样的常量也会进行扩展处理。 同时，该操作也可以在表达式中进行。例如，考虑以下表达式：

In [23]:
result = df.select(deviation=(bmi_expr - bmi_expr.mean()) / bmi_expr.std())
print(result)

shape: (4, 1)
┌───────────┐
│ deviation │
│ ---       │
│ f64       │
╞═══════════╡
│ 0.115645  │
│ -0.097471 │
│ -1.22912  │
│ 1.210946  │
└───────────┘


### with_columns
`with_columns`与`select`非常相似。

这两者的主要区别在于：`with_columns`会创建一个新的数据框，该数据框包含原始`DataFrame`中的列以及根据其输入表达式生成的新列；而`select`上下文仅包含由其输入表达式所选定的列。

In [24]:
result = df.with_columns(
    bmi=bmi_expr,
    avg_bmi=bmi_expr.mean(),
    ideal_max_bmi=25,
)
print(result)

shape: (4, 7)
┌────────────────┬────────────┬────────┬────────┬───────────┬───────────┬───────────────┐
│ name           ┆ birthdate  ┆ weight ┆ height ┆ bmi       ┆ avg_bmi   ┆ ideal_max_bmi │
│ ---            ┆ ---        ┆ ---    ┆ ---    ┆ ---       ┆ ---       ┆ ---           │
│ str            ┆ date       ┆ f64    ┆ f64    ┆ f64       ┆ f64       ┆ i32           │
╞════════════════╪════════════╪════════╪════════╪═══════════╪═══════════╪═══════════════╡
│ Alice Archer   ┆ 1997-01-10 ┆ 57.9   ┆ 1.56   ┆ 23.791913 ┆ 23.438973 ┆ 25            │
│ Ben Brown      ┆ 1985-02-15 ┆ 72.5   ┆ 1.77   ┆ 23.141498 ┆ 23.438973 ┆ 25            │
│ Chloe Cooper   ┆ 1983-03-22 ┆ 53.6   ┆ 1.65   ┆ 19.687787 ┆ 23.438973 ┆ 25            │
│ Daniel Donovan ┆ 1981-04-30 ┆ 83.1   ┆ 1.75   ┆ 27.134694 ┆ 23.438973 ┆ 25            │
└────────────────┴────────────┴────────┴────────┴───────────┴───────────┴───────────────┘


### filter

`filter`会根据一个或多个表达式来筛选数据框中的行。

In [25]:
result = df.filter(
    pl.col("birthdate").is_between(date(1982, 12, 31), date(1996, 1, 1)),
    pl.col("height") > 1.7,
)
print(result)

shape: (1, 4)
┌───────────┬────────────┬────────┬────────┐
│ name      ┆ birthdate  ┆ weight ┆ height │
│ ---       ┆ ---        ┆ ---    ┆ ---    │
│ str       ┆ date       ┆ f64    ┆ f64    │
╞═══════════╪════════════╪════════╪════════╡
│ Ben Brown ┆ 1985-02-15 ┆ 72.5   ┆ 1.77   │
└───────────┴────────────┴────────┴────────┘


### group_by和aggregations

在`group_by`这一语境中，数据行会根据分组表达式的唯一值进行分组。

在使用了`group_by`之后，我们使用`agg`来对各个组应用聚合表达式。

In [26]:
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
).agg(pl.col("name"))
print(result)

shape: (2, 2)
┌────────┬─────────────────────────────────┐
│ decade ┆ name                            │
│ ---    ┆ ---                             │
│ i32    ┆ list[str]                       │
╞════════╪═════════════════════════════════╡
│ 1990   ┆ ["Alice Archer"]                │
│ 1980   ┆ ["Ben Brown", "Chloe Cooper", … │
└────────┴─────────────────────────────────┘


由于在上述示例中我们仅指定了一个列的名称，所以我们会得到该列的各个组的列表形式。
我们可以根据需要设定任意数量的分组表达式，而`group_by`会根据所指定表达式中的不同值对行进行分组。在此示例中，我们按照出生年代以及个人身高是否低于 1.7米这两个因素进行组合分组。

In [27]:
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
    (pl.col("height") < 1.7).alias("short?"),
).agg(pl.col("name"))
print(result)

shape: (3, 3)
┌────────┬────────┬─────────────────────────────────┐
│ decade ┆ short? ┆ name                            │
│ ---    ┆ ---    ┆ ---                             │
│ i32    ┆ bool   ┆ list[str]                       │
╞════════╪════════╪═════════════════════════════════╡
│ 1980   ┆ false  ┆ ["Ben Brown", "Daniel Donovan"… │
│ 1980   ┆ true   ┆ ["Chloe Cooper"]                │
│ 1990   ┆ true   ┆ ["Alice Archer"]                │
└────────┴────────┴─────────────────────────────────┘


应用聚合表达式处理后生成的`DataFrame`中，每个聚合表达式都会对应一个列。

首先在左侧列出每个分组表达式，然后根据需要列出相应的列来表示聚合表达式的结果。接着，我们可以指定任意数量的聚合表达式:

In [28]:
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
    (pl.col("height") < 1.7).alias("short?"),
).agg(
    pl.len(),
    pl.col("height").max().alias("tallest"),
    pl.col("weight", "height").mean().name.prefix("avg_"),
)
print(result)

shape: (3, 6)
┌────────┬────────┬─────┬─────────┬────────────┬────────────┐
│ decade ┆ short? ┆ len ┆ tallest ┆ avg_weight ┆ avg_height │
│ ---    ┆ ---    ┆ --- ┆ ---     ┆ ---        ┆ ---        │
│ i32    ┆ bool   ┆ u32 ┆ f64     ┆ f64        ┆ f64        │
╞════════╪════════╪═════╪═════════╪════════════╪════════════╡
│ 1980   ┆ true   ┆ 1   ┆ 1.65    ┆ 53.6       ┆ 1.65       │
│ 1980   ┆ false  ┆ 2   ┆ 1.77    ┆ 77.8       ┆ 1.76       │
│ 1990   ┆ true   ┆ 1   ┆ 1.56    ┆ 57.9       ┆ 1.56       │
└────────┴────────┴─────┴─────────┴────────────┴────────────┘


由于表达式是惰性的，所以在将表达式置于某个上下文中使用时，Polars会尝试在执行数据转换操作之前简化该表达式。在一个执行环境中，相互独立的表达式显然是可以并行处理的，Polars 会充分利用这一点，同时在使用表达式扩展时也会对表达式执行进行并行化处理。

而当使用 Polars 的惰性 API时(接下来会介绍)，还能进一步获得性能提升。

### 惰性API（Lazy API）

Polars 支持两种运行模式：惰性模式（Lazy API）和主动模式 (Eager API) 。

到目前为止的示例均使用了主动API，在这种模式下，查询会立即执行。

而在惰性模式中，只有在收集查询结果时才会对其进行评估。将执行操作推迟到最后一刻能够带来显著的性能优势，这也是为什么在大多数情况下，惰性 API更为被青睐。让我们通过一个示例来说明这一点:

In [29]:
# 使用主动API方式处理数据
df = pl.read_csv("output.csv")
df_filtered = df.filter(pl.col("age") > 30)
df_agg = df_filtered.group_by("city").agg(pl.col("salary").mean())

print(df_agg)

shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Tokyo  ┆ 8000.0 │
│ Berlin ┆ 9000.0 │
│ Paris  ┆ 7000.0 │
└────────┴────────┘


在本例中，我们使用主动API来：
- 读取数据
- 过滤年龄大于30的记录
- 按城市分组并计算平均薪资 

每一步操作都会立即执行，并返回中间结果。这可能会造成极大的浪费，因为我们可能会进行不必要的工作或加载未被使用的额外数据。如果我们改为使用延迟式 API，并等到所有步骤都定义好后再执行，那么查询规划器就能够进行各种优化。在这种情况下: 
- 谓词下推：在读取数据集时尽早应用过滤条件，这样就只读取年龄大于30的行。
- 投影下推：在读取数据集时仅选择所需的列，从而无需加载额外的列。

In [30]:
# 使用惰性API方式处理数据
q = (
    pl.scan_csv("output.csv")  
    .filter(pl.col("age") > 30)  
    .group_by("city")  
    .agg(pl.col("salary").mean())  
)

# 执行查询并获取结果
df = q.collect()
print(df)

shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Paris  ┆ 7000.0 │
│ Berlin ┆ 9000.0 │
│ Tokyo  ┆ 8000.0 │
└────────┴────────┘


这些操作将显著减轻内存和 CPU的负担，从而能够将更大的数据集存储在内存中并更快地进行处理。

一旦定义好查询，我们就调用`collect`来通知Polars想要执行该查询。

#### 何时使用惰性API

一般来说，应优先使用惰性API，除非我们对中间结果感兴趣，或者正在进行探索性研究并且还不清楚最终查询结果是什么样子。

## 流式（Steaming）

惰性 API的另一个优点是它允许以流式方式执行查询。与一次性处理所有数据不同。Polars可以分批执行查询，使得当数据集无法全部装入内存时，也能处理。此外，与Polars 的内存引擎相比，流式引擎的性能也更出色。

要告知 Polars 我们希望以流式模式执行查询，需要在代码中设置`engine='streaming'`这个参数。示例如下：

In [31]:
q = (
    pl.scan_csv("output.csv")  
    .filter(pl.col("age") > 30)  
    .group_by("city")  
    .agg(pl.col("salary").mean())  
)

# 执行流式查询并获取结果
df = q.collect(engine="streaming")

## 读写数据

Polars 支持读取和写入常见的文件格式（例如 csv、json、parquet）、云存储（S3、Azure Blob、BigQuery）以及数据库（例如 postgres、mysql）。

使用`read_csv` 函数读取 csv 文件,使用`write_csv` 函数写入 csv 文件。

In [32]:

# 写入 CSV 文件
df.write_csv("output.csv")

# 读取 CSV 文件
df_from_csv = pl.read_csv("output.csv")
print(df_from_csv)

# 读写 Parquet 文件（更高效）
df.write_parquet("output.parquet")
df_from_parquet = pl.read_parquet("output.parquet")
print(df_from_parquet)

# 读写 JSON 文件
df.write_json("output.json")
df_from_json = pl.read_json("output.json")
print(df_from_json)


shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Paris  ┆ 7000.0 │
│ Berlin ┆ 9000.0 │
│ Tokyo  ┆ 8000.0 │
└────────┴────────┘
shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Paris  ┆ 7000.0 │
│ Berlin ┆ 9000.0 │
│ Tokyo  ┆ 8000.0 │
└────────┴────────┘
shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Paris  ┆ 7000.0 │
│ Berlin ┆ 9000.0 │
│ Tokyo  ┆ 8000.0 │
└────────┴────────┘


## 与 pandas 对比
性能对比

In [33]:
import time
import numpy as np
import pandas as pd

# 创建大型数据集
n_rows = 1_000_000
data = {
    "x": np.random.randn(n_rows),
    "y": np.random.randn(n_rows),
    "category": np.random.choice(["A", "B", "C"], n_rows)
}

# 创建 polars DataFrame
df_polars = pl.DataFrame(data)

# 创建 pandas DataFrame
df_pandas = pd.DataFrame(data)

# 测量聚合操作的时间
start_time = time.time()
result_polars = df_polars.group_by("category").agg(pl.col("x").mean())
end_time = time.time()
polars_time = end_time - start_time

start_time = time.time()
result_pandas = df_pandas.groupby("category")["x"].mean()
end_time = time.time()
pandas_time = end_time - start_time

print(f"polars 处理时间: {polars_time:.3f} 秒")
print(f"pandas 处理时间: {pandas_time:.3f} 秒")
print(f"速度提升: {pandas_time/polars_time:.1f} 倍")

polars 处理时间: 0.020 秒
pandas 处理时间: 0.107 秒
速度提升: 5.3 倍


内存使用对比

In [34]:
import sys

# 查看内存使用
polars_memory = sys.getsizeof(df_polars)
pandas_memory = sys.getsizeof(df_pandas)

print(f"polars 内存使用: {polars_memory / 1024 / 1024:.2f} MB")
print(f"pandas 内存使用: {pandas_memory / 1024 / 1024:.2f} MB")
print(f"内存节省: {pandas_memory/polars_memory:.1f} 倍")

polars 内存使用: 0.00 MB
pandas 内存使用: 70.57 MB
内存节省: 1321431.5 倍


### 与 pandas 互操作
polars可以轻松与 pandas进行互操作：

In [35]:
# 转换为 pandas DataFrame
pandas_df = df.to_pandas()
print(type(pandas_df))
print(pandas_df.head())

# 从 pandas DataFrame 创建
df_from_pandas = pl.from_pandas(pandas_df)
print(type(df_from_pandas))
print(df_from_pandas.head())

<class 'pandas.core.frame.DataFrame'>
     city  salary
0   Paris  7000.0
1  Berlin  9000.0
2   Tokyo  8000.0
<class 'polars.dataframe.frame.DataFrame'>
shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Paris  ┆ 7000.0 │
│ Berlin ┆ 9000.0 │
│ Tokyo  ┆ 8000.0 │
└────────┴────────┘


### 高级功能

表达式 API

polars的表达式 API 提供了强大的数据操作能力：

In [36]:
# 使用表达式进行复杂操作
result = df.select([
    pl.col("name"),
    pl.col("age"),
    pl.col("salary"),
    (pl.col("salary") / pl.col("age")).alias("salary_per_age"),
    pl.when(pl.col("age") > 30)
      .then(pl.col("salary") * 1.1)
      .otherwise(pl.col("salary"))
      .alias("adjusted_salary")
])
print(result)

ColumnNotFoundError: unable to find column "name"; valid columns: ["city", "salary"]

时间序列处理

In [ ]:
# 创建时间序列数据
date_ranges = pl.date_range(
    start=pl.datetime(2023, 1, 1),
    end=pl.datetime(2023, 1, 10),
    interval="1d",
    eager=True
)

ts_df = pl.DataFrame({
    "date": date_ranges,
    "value": range(1, 11)
})
print(ts_df)

# 时间序列操作
ts_result = ts_df.with_columns([
    pl.col("value").diff().alias("diff"),
    pl.col("value").shift(1).alias("lag"),
    pl.col("value").rolling_mean(window_size=3).alias("rolling_mean")
])
print(ts_result)

shape: (10, 2)
┌────────────┬───────┐
│ date       ┆ value │
│ ---        ┆ ---   │
│ date       ┆ i64   │
╞════════════╪═══════╡
│ 2023-01-01 ┆ 1     │
│ 2023-01-02 ┆ 2     │
│ 2023-01-03 ┆ 3     │
│ 2023-01-04 ┆ 4     │
│ 2023-01-05 ┆ 5     │
│ 2023-01-06 ┆ 6     │
│ 2023-01-07 ┆ 7     │
│ 2023-01-08 ┆ 8     │
│ 2023-01-09 ┆ 9     │
│ 2023-01-10 ┆ 10    │
└────────────┴───────┘
shape: (10, 5)
┌────────────┬───────┬──────┬──────┬──────────────┐
│ date       ┆ value ┆ diff ┆ lag  ┆ rolling_mean │
│ ---        ┆ ---   ┆ ---  ┆ ---  ┆ ---          │
│ date       ┆ i64   ┆ i64  ┆ i64  ┆ f64          │
╞════════════╪═══════╪══════╪══════╪══════════════╡
│ 2023-01-01 ┆ 1     ┆ null ┆ null ┆ null         │
│ 2023-01-02 ┆ 2     ┆ 1    ┆ 1    ┆ null         │
│ 2023-01-03 ┆ 3     ┆ 1    ┆ 2    ┆ 2.0          │
│ 2023-01-04 ┆ 4     ┆ 1    ┆ 3    ┆ 3.0          │
│ 2023-01-05 ┆ 5     ┆ 1    ┆ 4    ┆ 4.0          │
│ 2023-01-06 ┆ 6     ┆ 1    ┆ 5    ┆ 5.0          │
│ 2023-01-07 ┆ 7     ┆ 1    ┆ 

## 总结
polars是一个强大且高效的数据处理库，具有以下优势：

- 出色的性能，尤其适合处理大型数据集

- 内存效率高，支持惰性执行

- 丰富的数据操作功能

- 良好的生态系统集成

对于需要处理大规模数据的应用场景，polars是一个很好的选择。对于熟悉 pandas的用户，polars的学习曲线相对平缓，大部分概念和操作都是相通的。

#### 何时选择 polars 而不是 pandas：

- 处理大型数据集（GB 级别或更大）

- 需要最佳性能

- 内存受限的环境

- 需要多线程处理

#### 何时选择 pandas 而不是 polars：

- 需要与丰富的 pandas 生态系统集成

- 使用许多专门的数据分析库

- 处理小型数据集且开发速度更重要

更多详细信息和高级用法，请参考 [polars 官方文档](https://pola.rs/)